In [1]:
import pandas as pd

In [8]:
abpred_database = pd.read_csv('../data/Data_skempi_ab-bind_Final-merge.csv',index_col=0)
abpred_traindata =  pd.read_csv("../data/abpred_train_2020_v2.csv",index_col=0)

abpred_traindata.reset_index(inplace=True)
abpred_database.reset_index(drop=True,inplace=True)

In [9]:
# fix lowercase mutant nomeclature on some mutants
abpred_database["mutant"] = abpred_database["mutant"].str.upper()

abpred_database["abpred_mutant"] = abpred_database["#PDB"]+"."+abpred_database["chain"]+abpred_database["mutant"]
abpred_traindata["abpred_mutant"] = abpred_traindata["index"].str.rsplit(".",4,expand=True)[0]

In [10]:
# Some mutant were duplicated betwen skempi and abbind database 
abpred_traindata.drop_duplicates(inplace=True)

In [11]:
#abpred_traindata.sort_values(by="abpred_mutant").iloc[100:150]
abpred_database["ddG"].isna().value_counts()

False    937
Name: ddG, dtype: int64

In [12]:
traindata_mut = abpred_traindata[(abpred_traindata["index"].str.split(".").str.len()==6)].copy()
traindata_wt = abpred_traindata[(abpred_traindata["index"].str.split(".").str.len()==8)].copy()

traindata_wt["abpred_mutant"] = traindata_wt["abpred_mutant"].str.rsplit(".",2,expand=True)[0]

In [30]:
traindata_mut2 = traindata_mut.merge(abpred_database[["abpred_mutant","ddG"]],how="left")

In [32]:
traindata_wt2 = traindata_wt.merge(abpred_database[["abpred_mutant","ddG"]],how="left")
traindata_wt2["ddG"] = -traindata_wt2["ddG"]

In [35]:
abpred_traindata_final = pd.concat([traindata_mut2,traindata_wt2])
abpred_traindata_final.drop("abpred_mutant",axis=1,inplace=True)

In [36]:
abpred_traindata_final.set_index("index",inplace=True)

In [37]:
abpred_traindata_final.ddG.isna().value_counts()

False    1874
Name: ddG, dtype: int64

In [47]:
# Drop strucures wiht abs ddG value equal 8
abpred_traindata_final = abpred_traindata_final[-(abpred_traindata_final.ddG.abs() == 8.)]

In [49]:
abpred_traindata_final.to_csv("../data/abpred_train_2020_v2.csv")